This is a simple tutorial to map a model to the AKIDA1000 hardware

In [6]:
import tensorflow as tf
from tensorflow import keras
from quantizeml.models import quantize
from cnn2snn import convert
import os
import datetime
import tensorflow as tf
from tensorflow import keras
import cnn2snn
import akida as ak
from quantizeml.models import quantize
from quantizeml.layers import QuantizationParams


Setting the akida env version to v1 is very important to port it to hardware

In [7]:
os.environ["CNN2SNN_TARGET_AKIDA_VERSION"] = "v1"
print('----------------------------------')
print("    TF version: ", tf.__version__)
print("MetaTF version: ", ak.__version__)
print(' Akida version: ', cnn2snn.get_akida_version()) 
print('----------------------------------')

!which python

today = datetime.datetime.now()
todayAsStr = str(today.year) + '-' + str(today.month) + '-' + str(today.day) + '_' + str(today.hour) + '-' + str(today.minute)
print('Time of run: ', todayAsStr)

----------------------------------
    TF version:  2.15.0
MetaTF version:  2.11.0
 Akida version:  AkidaVersion.v1
----------------------------------
/home/kannan/anaconda3/envs/tf_env/bin/python
Time of run:  2025-11-10_17-30


In [9]:
#!/usr/bin/env python3
"""
Rebuild ESA ConvNet and load weights (TF 2.15 compatible),
then cut off the final Reshape to get a flat (2280,) output.
"""

import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model

# --- Architecture (same as your training script) ---
def build_sequential_model(window, n_channels, forecast_horizon=None):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(window, n_channels, 1)),
        tf.keras.layers.Conv2D(16, (5,5), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),

        tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),

        tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),

        tf.keras.layers.Conv2D(64, (1,1), padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),

        tf.keras.layers.Dense(forecast_horizon * n_channels, activation='linear', name="dense_out"),
        tf.keras.layers.Reshape((forecast_horizon, n_channels), name="reshape_out"),
    ], name="akida_friendly_cnn")
    return model

# --- Params (match training) ---
WINDOW = 60
N_CHANNELS = 76
FORECAST = 30
WEIGHTS_PATH = "ESA_cnn_simon.weights.h5"   # your weights file path

# --- Build + load ---
base_model = build_sequential_model(WINDOW, N_CHANNELS, FORECAST)
base_model.load_weights(WEIGHTS_PATH)
print("\n✅ Base model + weights loaded successfully!")
base_model.summary()

# --- Cut off the final Reshape layer, keep flat Dense(2280) output ---
# Last two layers are: Dense(..., name="dense_out") then Reshape(..., name="reshape_out")
flat_output = base_model.get_layer("dense_out").output  # shape: (None, 2280)
flat_model = Model(inputs=base_model.input, outputs=flat_output, name="akida_friendly_cnn_flat")

print("\n🧩 New model without final Reshape:")
flat_model.summary()

# Optional: compile (use any metrics you need)
flat_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanAbsoluteError(),
             tf.keras.metrics.RootMeanSquaredError()]
)

# Quick forward test
x_dummy = np.zeros((1, WINDOW, N_CHANNELS, 1), dtype=np.float32)
y_pred_flat = flat_model(x_dummy)
print("Flat output shape:", y_pred_flat.shape)  # expect (1, 30*76) = (1, 2280)

# (Optional) Save this trimmed model for quantization/compat checks
# flat_model.save("ESA_cnn_simon_flat_savedmodel")  # SavedModel folder
# flat_model.save_weights("ESA_cnn_simon_flat.weights.h5")



✅ Base model + weights loaded successfully!
Model: "akida_friendly_cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 60, 76, 16)        416       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 30, 38, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 30, 38, 32)        4640      
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 15, 19, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 15, 19, 64)        18496     
                                                                 
 ma

In [ ]:
# # 1. Load trained model 
# model_src = tf.keras.models.load_model("ESA_cnn_simon.h5", compile=False)
# model_src.summary()


In [ ]:
#!/usr/bin/env python3
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model


def build_flat_bounded_model(window, n_channels, forecast_horizon):

    ReLU6 = lambda name=None: tf.keras.layers.ReLU(max_value=6.0, name=name)
    return tf.keras.Sequential([
        tf.keras.layers.Input(shape=(window, n_channels, 1), name="input"),
        tf.keras.layers.Conv2D(16, (5,5), padding='same', name="conv1"),
        ReLU6("relu1"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same', name="pool1"),

        tf.keras.layers.Conv2D(32, (3,3), padding='same', name="conv2"),
        ReLU6("relu2"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same', name="pool2"),

        tf.keras.layers.Conv2D(64, (3,3), padding='same', name="conv3"),
        ReLU6("relu3"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same', name="pool3"),

        tf.keras.layers.Conv2D(64, (1,1), padding='same', name="conv4"),
        ReLU6("relu4"),

        tf.keras.layers.Flatten(name="flatten"),
        tf.keras.layers.Dense(64, name="fc1"),
        ReLU6("relu_fc1"),

        # Flat output (no reshape) — safer for quantizers
        tf.keras.layers.Dense(forecast_horizon * n_channels, activation='linear', name="dense_out"),
    ], name="akida_friendly_cnn_flat_relu6")

# --- Params (match training) ---
WINDOW    = 60
N_CHANNEL = 76
FORECAST  = 30
WEIGHTS   = "ESA_cnn_simon.weights.h5"   # your weights file


base = build_flat_bounded_model(WINDOW, N_CHANNEL, FORECAST)

try:
    base.load_weights(WEIGHTS)
except Exception as e:
    print("load_weights issue:", e)

x_dummy = np.zeros((1, WINDOW, N_CHANNEL, 1), dtype=np.float32)
y = base(x_dummy)
print("Output shape (should be 2280):", y.shape)




✅ Weights loaded into bounded-ReLU flat model.
Output shape (should be 2280): (1, 2280)


In [14]:
compatibilityIssues = cnn2snn.check_model_compatibility(base, device=ak.AKD1000())
print('Compatibility issues: ', compatibilityIssues)


1/1 [==============================] - 0s 302ms/step
Compatibility issues:  None


In [ ]:
from quantizeml.models import quantize
from quantizeml.layers import QuantizationParams

# Quantize the model
qparams = QuantizationParams(input_weight_bits=8, weight_bits=4, activation_bits=4, per_tensor_activations=True)
model_quantized = quantize(model_src, qparams=qparams)

In [ ]:
print(' Akida version check: ', cnn2snn.get_akida_version()) 
akida_model = cnn2snn.convert(model_quantized, file_path='akida_model.fbz')

In [ ]:
akida_model = cnn2snn.convert(model_quantized, file_path='akida_model.fbz')

In [ ]:
print('Model version: ', akida_model.ip_version, '\n')